In [60]:
from rlcube.models.models import DNN
from rlcube.envs.cube2 import Cube2
import numpy as np
import torch

net = DNN()
net.load("models/model_best.pth")
net.eval()

DNN(
  (fc_in): Linear(in_features=144, out_features=512, bias=True)
  (residual_blocks): ModuleList(
    (0-3): 4 x ResidualBlock(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=512, out_features=1024, bias=True)
      (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc2): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
  (fc_value): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc_policy): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=12, bias=True)
  )
)

In [61]:
env = Cube2()
obs, _ = env.reset()
obs1, _, _, _, _ = env.step(0)
obs2, _, _, _, _ = env.step(0)
obs3, _, _, _, _ = env.step(2)
obs4, _, _, _, _ = env.step(2)
for _ in range(10):
    obsMany, _, _, _, _ = env.step(env.action_space.sample())
batched_obs = torch.tensor(np.array([obs, obs1, obs2, obs3, obs4, obsMany]), dtype=torch.float32)
out = net(batched_obs)
print(out["value"])
print(torch.argmax(out["policy"], dim=1))

tensor([[ 0.0166],
        [ 1.0147],
        [ 1.1610],
        [ 0.9844],
        [-0.0268],
        [ 1.1526]], grad_fn=<AddmmBackward0>)
tensor([10,  1,  5,  0, 10,  1])
